In [ ]:
# Import all libraries
import rasterio
#import rasterio.plot
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from glob import glob
import os
import sys

# Infrared Percentage Vegetation Index (IPVI) description
### Reference: Crippen, R.E., 1990. Calculating the vegetation index faster. Remote Sensing of Environment 34, 71–73.


### General description
-This code has been tested with Sentinel-2B MSI Level 2 satellite data. 
-Any Sentinel-2B and 2A MSI Level 2 data can be download from https://scihub.copernicus.eu/dhus/#/home  
-Due to three different spectral resolution of Sentinel-2 data (10m, 20m, and 60m), a preprocessing as resmapling has been applied to make all bands with 10m resolution. The resampling procedure has been done by SNAP ESA software.
-The resampled data has been subset by the SANP ESA software to reduce the data memory.
-Resampling and subset can also be done by python API of SNAP called snappy. Details on use sanppy API can be found here: https://senbox.atlassian.net/wiki/spaces/SNAP/pages/19300362/How+to+use+the+SNAP+API+from+Python
-Resampling with snappy is not straight forward, due to  
-For understanding the band with specific radiometric resolution, see here https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/resolutions/radiometric

In [ ]:
### open band 8 with central wavelength of approx. 832.9 nm or IR or Infrared band, 
### for Sentinel-2A Band 8 is 832.8 nm
### for Sentinel-2B Band 8 is 832.9 nm
ras1 = 'YOUR:/file_location/S2B_MSIL2A_20200708T102559_N0214_R108_*****_20200708T135942_resampled.data/B8.img'

In [ ]:
### this part is not necessary, unless if someone would like to see this info
with rasterio.open(ras1) as src1:
    print(src1.profile) # print the profile of raster data
    print(src1.count) # count the number of bands
    print(src1.crs) # show the raster geocoordinate
    print(src1.crs.wkt) #show the raster WKT

In [ ]:
### read band as 2D numpy array
with rasterio.open(ras1, 'r') as ras1_src:
    print(ras1_src)
    ras1_arr = ras1_src.read(1)
    print(ras1_arr.shape)
    print(ras1_arr)    
### remove no data value from array
ndv = 0
ras1_arr_ndv = ma.masked_where(ras1_arr == ndv, ras1_arr)
print(ras1_arr_ndv.shape)
print(ras1_arr_ndv)
print(np.min(ras1_arr_ndv))
print(np.max(ras1_arr_ndv))

In [ ]:
### open band 4 with central wavelength of approx. 560 nm or Red visible band,
### for Sentinel-2A Band 4 is 664.6 nm
### for Sentinel-2B Band 4 is 664.9 nm
ras2 = 'YOUR:/file_location/S2B_MSIL2A_20200708T102559_N0214_R108_*****_20200708T135942_resampled.data/B4.img'

In [ ]:
### this part is not necessary, unless if someone would like to see this info
with rasterio.open(ras2) as src2:
    print(src2.profile) # print the profile of raster data
    print(src2.count) # number of bands
    print(src2.crs) # show the raster geocoordinate
    print(src2.crs.wkt) #show the raster WKT

In [ ]:
### read band as 2D numpy array
with rasterio.open(ras2, 'r') as ras2_src:
    print(ras2_src)
    ras2_arr = ras2_src.read(1)
    print(ras2_arr.shape)
    print(ras2_arr)
### remove no data value from array
ndv = 0
ras2_arr_ndv = ma.masked_where(ras2_arr == ndv, ras2_arr)
print(ras2_arr_ndv.shape)
print(ras2_arr_ndv)
print(np.min(ras2_arr_ndv))
print(np.max(ras2_arr_ndv))

In [ ]:
### IPVI calculation
print("Calculating IPVI.....")
IPVI = (ras1_arr_ndv*0.0001) / ((ras2_arr_ndv*0.0001) + (ras1_arr_ndv*0.0001))
# multiply 0.0001 to convert 16-bit int to 32-bit float
# because the Sentinel-2 Bottom of Atmosphere reflectance values are quantified by 10000
# information can be found in the metadata: Level-2A_DataStrip_ID>
# Image_Data_Info>Radiometric_info>QUANTIFICATION_VALUES_LIST

print(IPVI) # to see the calculated 2D numpy array
print(IPVI.shape) # to see the row and column of calculated array
print(np.min(IPVI)) # minimum value
print(np.max(IPVI)) # maximum value
### plot array
plt.imshow(IPVI)
### plot colorbar
plt.colorbar(shrink=0.5)
plt.clim(np.min(IPVI), np.max(IPVI))
### add tile
plt.title('IPVI {}'.format(IPVI.shape))
### add x and y label
plt.xlabel('Column #')
plt.ylabel('Row #')
### make the output file directory
fig_direc = 'YOUR:/output_file_directory/IPVI'
if not os.path.exists(fig_direc):
    os.makedirs(fig_direc)
### Save the figure as jpg format
plt.savefig('YOUR:/output_file_directory/IPVI/IPVI.jpg', dpi=600) # dpi can be changed according to needs but 600 is high 
# enough and such good resolution

### Save calculated index as GeoTiff file
### Get metadata from raster 2
with rasterio.open(ras2) as src:
    meta = src.meta
meta.update(dtype=rasterio.float32)

### Create output folder and write output file in it as geotiff
IPVI_file = "YOUR:/output_file_directory/IPVI/IPVI.tif"
os.makedirs(os.path.dirname(IPVI_file), exist_ok=True)
with rasterio.open(IPVI_file, 'w', **meta) as dst:
    dst.write(IPVI.astype(rasterio.float32), 1)